# Agentic & MCP Demo



## Setup

### Pre-Requisites

Before starting, ensure you have the following:
- A running Llama Stack server with the mcp::crm toolgroup configured

### Installing dependencies

This code requires `llama-stack` and the `llama-stack-client`, both at version `0.2.2`. Lets begin by installing them:

In [1]:
!pip3 install llama-stack-client==0.2.2 llama-stack==0.2.2 --quiet


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


### Configuring logging

Now that we have our dependencies, lets setup logging for the application:

In [2]:
from llama_stack_client.lib.agents.event_logger import EventLogger

import logging

logger = logging.getLogger(__name__)
if not logger.hasHandlers():  
    logger.setLevel(logging.INFO)
    stream_handler = logging.StreamHandler()
    stream_handler.setLevel(logging.INFO)
    formatter = logging.Formatter('%(message)s')
    stream_handler.setFormatter(formatter)
    logger.addHandler(stream_handler)

### Connecting to llama-stack server

For the llama-stack instance, you can either run it locally or connect to a remote llama-stack instance.


In [3]:
import sys
import os
# Add project root (assumes notebook is in ./notebooks)
sys.path.append(os.path.abspath(".."))
print("sys.path now includes:", sys.path[-1])
from dotenv import load_dotenv
from scripts.utils import get_any_available_model
load_dotenv()

# base_url = os.getenv("REMOTE_BASE_URL", "http://localhost:5001")

base_url="http://localhost:8321"

from llama_stack_client import LlamaStackClient

client = LlamaStackClient(
    base_url=base_url
)
model = get_any_available_model(client)
# model = 'granite-3-8b-instruct'

print(model)
    
logger.info(f"Connected to Llama Stack server @ {base_url} \n")

Connected to Llama Stack server @ http://localhost:8321 



sys.path now includes: /opt/app-root/src/rh-summit-agentic-demo
Qwen/Qwen3-14B


### Validate tools are available in our llama-stack instance

When an instance of llama-stack is redeployed your tools need to re-registered. Also if a tool is already registered with a llama-stack instance, if you try to register one with the same `toolgroup_id`, llama-stack will throw you an error.

For this reason it is recommended to include some code to validate your tools and toolgroups. This is where the `mcp_url` comes into play. The following code will check that  the `builtin::crm` toolgroups are registered as tools.

In [4]:
registered_tools = client.tools.list()
registered_toolgroups = [t.toolgroup_id for t in registered_tools]
logger.info(f"Your Llama Stack server is already registered with the following tool groups @ {set(registered_toolgroups)} \n")

Your Llama Stack server is already registered with the following tool groups @ {'mcp::slack', 'mcp::pdf', 'builtin::code_interpreter', 'mcp::crm', 'builtin::websearch', 'builtin::rag', 'builtin::wolfram_alpha'} 



## Get active opportunities

This is a simple example to get a list of active opportunities from the CRM system.

### System Prompts for different models

**Note:** If you have multiple models configured with your Llama Stack server, you can choose which one to run your queries against. When switching to a different model, you may need to adjust the system prompt to align with that model’s expected behavior. Many models provide recommended system prompts for optimal and reliable outputs—these are typically documented on their respective websites.

In [5]:
# Here is a system prompt we have come up with which works well for this query

sys_prompt1= """You are a helpful assistant. Use tools to answer. When you use a tool always respond with a summary of the result."""

In [6]:
print(model)
from llama_stack_client import Agent
# Create simple agent with tools
agent = Agent(
    client,
    model=model, # replace this with your choice of model
    instructions = sys_prompt1 , # update system prompt based on the model you are using
    tools=["mcp::crm"],
    tool_config={"tool_choice":"auto"},
    sampling_params={
        "max_tokens":4096,
        "strategy": {"type": "greedy"},
    }
)

user_prompts = ["Get active opportunities for acme"]
session_id = agent.create_session(session_name="crm_demo")

for prompt in user_prompts:
    turn_response = agent.create_turn(
        messages=[
            {
                "role":"user",
                "content": prompt
            }
        ],
        session_id=session_id,
        stream=True,
    )
    for log in EventLogger().log(turn_response):
        log.print()

Qwen/Qwen3-14B
inference> <think>
Okay, the user is asking for active opportunities for Acme. Let me check the tools available. There's a function called getOpportunities that retrieves active opportunities from the CRM. The parameters are empty, so I don't need any arguments. I'll call that function to get the list. I should make sure to use the correct JSON structure within the tool_call tags.
</think>

<tool_call>
{"name": "getOpportunities", "arguments": {}}
</tool_call>


### Output Analysis

This script defines and runs a simple agent using Llama Stack that is equipped with CRM tools (mcp::crm). It performs a single action: retrieves one or more active opportunities using a user prompt. The response is streamed, and each part of the process (model inference, tool call, and results) is logged step-by-step.

Output: 

The model infers that it should call the getOpportunities tool with argument id=1.

This suggests the tool is expected to return the active opportunity with ID 1.

The tool returns a JSON-like structure containing multiple active opportunities.

The model interprets the output and summarizes only one opportunity (ID 1), including:

Account: Acme Corp

Description: "Upsell - Cloud package"

Amount: $5,000

Year: 2025


## Analyze account sentiment for active opportunities

Get a list of active opportunities, then find the associated support cases for each opportunity, then anayse the mood of the account.

In [7]:
# Here is a system prompt we have come up with which works well for this query
sys_prompt2="""You are a helpful AI assistant, responsible for helping me find and communicate information back to my team.
    You have access to a number of tools.
    Whenever a tool is called, be sure return the Response in a friendly and helpful tone.
    When you are asked to find out about opportunities and support casescounts you must use a tool.
    If you need to create a pdf, add this markdown to the start of the content:  ![ParasolCloud Logo](https://i.postimg.cc/MHZB5tmL/Screenshot-2025-04-21-at-5-58-46-PM.png)
*Secure Cloud Solutions for a Brighter Business*  
---
    """

In [8]:
from llama_stack_client.lib.agents.agent import Agent

agent = Agent(
    client=client,
    model=model,
    instructions=sys_prompt2,
    tools=["mcp::crm", "mcp::pdf"],
    tool_config={"tool_choice":"auto"},
    sampling_params={
        "max_tokens":4096,
        "strategy": {"type": "greedy"},
    }
)

session_id = agent.create_session(session_name="active_opportunity_and_account_health")

prompts = [
    """Get active opportunities for ACME""",
    """get a list of support cases for the account id""",
    """Analyze the support cases and determine how happy the account is""",
     """Create a pdf with the content 'hello world'""",

]
for i, prompt in enumerate(prompts):    
    turn_response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id,
        stream=True,
    )
    logger.info(f"========= Turn: {i} =========")
    for log in EventLogger().log(turn_response):
        log.print()


========= Turn: 0 =========


inference> <think>
Okay, the user is asking for active opportunities for ACME. Let me check the tools available. There's a function called getOpportunities that retrieves active opportunities from the CRM. The parameters are empty, so I don't need any arguments. I should call that function. Wait, does the function require any parameters? The tool definition says properties and required are empty, so no. Just call getOpportunities. Then, once I get the data, I can present it to the user. But the user specifically mentioned ACME. Hmm, the function doesn't take parameters, so maybe the filtering is done on the server side. I'll proceed to call getOpportunities and then check the results. If the results include ACME's opportunities, great. If not, maybe the user needs to specify more details. But according to the tools provided, I can't add parameters. So I'll go ahead and call the function.
</think>

<tool_call>
{"name": "getOpportunities", "arguments": {}}
</tool_call>


========= Turn: 1 =========


inference> <think>
Okay, the user is asking for a list of support cases for a specific account ID. Let me check the available tools. There's the getSupportCases function, but looking at its parameters, it doesn't mention anything about an account ID. The function's parameters are empty, so it probably retrieves all support cases or requires some other filtering method. Since the user specified an account ID, but the function doesn't accept parameters, I can't filter by account ID. I should inform the user that the current tool doesn't support filtering by account ID and offer to retrieve all support cases instead. Alternatively, maybe there's a way to use another function or adjust the query. Wait, the tools provided don't have any other functions related to support cases. So the best approach is to let the user know that the getSupportCases function doesn't take an account ID parameter and ask if they want all cases or if there's another way to assist them.
</think>

The function `get

========= Turn: 2 =========


inference> <think>
Okay, the user wants me to analyze support cases and determine how happy the account is. But first, I need to get the list of support cases. However, the getSupportCases function doesn't take an account ID parameter. The user mentioned "for the account id," but they didn't specify which account ID. Wait, in the previous message, the user said "get a list of support cases for the account id," but the function can't filter by account ID. So maybe they want all support cases, but then how do they relate to the account?

Wait, the user's latest query is to analyze support cases to determine account happiness. But without knowing which account, I can't filter the cases. The function as it stands can't take an account ID. So I need to ask the user to clarify which account they're referring to. Alternatively, maybe they want to analyze all support cases, but that might not be specific to an account. 

Alternatively, maybe the user expects that the support cases are already 

========= Turn: 3 =========


inference> <think>
Okay, the user wants to create a PDF with the content 'hello world'. Let me check the available tools. There's the create_pdf_from_markdown function. The parameters required are markdown, outputFilename, paperFormat, paperOrientation, paperBorder, and watermark. The user provided the markdown content as 'hello world', but the other parameters aren't specified. I need to use default values for the missing ones. The output filename wasn't given, so maybe I can suggest a default like 'hello_world.pdf'. Paper format defaults to letter, orientation to portrait, border to 2cm, and no watermark. I should structure the tool call with these defaults. Let me make sure all required parameters are included. Yes, the function requires all of them. Alright, I'll generate the tool call with the provided markdown and default values for the rest.
</think>

<tool_call>
{"name": "create_pdf_from_markdown", "arguments": {"markdown": "hello world", "outputFilename": "hello_world.pdf", "p

#### Output Analysis

`tool_execution> Tool:getSupportCases Args:{'id': '1'}`

The model identifies account_id: 1 (from the previous opportunity) and passes it to the getSupportCases tool.

<b>Tool Response:</b>
Returns 3 support cases for Acme Corp, each with detailed metadata.

<b>Model's Inference:</b>
Extracts and formats all 3 support cases clearly.

Each case is displayed with its:

* Subject
* Description
* Status
* Severity
* Created timestamp

The agent performs a sentiment analysis over the support cases to determine the state of the account and makes recommendations.


## Key Takeaways

This tutorial demonstrates how to build agentic MCP applications with Llama Stack. We do so by initializing an agent while giving it access to the MCP tools configured with Llama Stack, then invoking the agent on each of the specified queries. 

In [9]:
from llama_stack_client import Agent
# Create simple agent with tools
sys_prompt1= """You are a helpful assistant. Use tools to answer."""
agent = Agent(
    client,
    model="Qwen/Qwen3-14B", # replace this with your choice of model
    instructions = sys_prompt1 , # update system prompt based on the model you are using
    tools=["mcp::python"],
    tool_config={"tool_choice":"auto"},
    sampling_params={
        "max_tokens":10000,
        "strategy": {"type": "greedy"},
    }
)

user_prompts = ["Find the number of letter s in the word strawberry"]
session_id = agent.create_session(session_name="OCP_demo")

for prompt in user_prompts:
    turn_response = agent.create_turn(
        messages=[
            {
                "role":"user",
                "content": prompt
            }
        ],
        session_id=session_id,
        stream=True,
    )
    for log in EventLogger().log(turn_response):
        log.print()

400: Invalid value: Toolgroup mcp::python not found, available toolgroups: builtin::code_interpreter, builtin::rag, builtin::websearch, builtin::wolfram_alpha, mcp::crm, mcp::pdf, mcp::slack
